In [1]:
!pip install osmnx torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.5 MB/s eta 0:00:0000:01


In [4]:
import sys
import math
import time
import pickle
import numpy as np
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_geometric.nn.inits import reset, uniform, zeros
from torch_geometric.utils import from_networkx
from sklearn.metrics import f1_score, average_precision_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from tqdm import tqdm
from typing import Union, Tuple, Callable, Optional

# Thiết lập device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ==========================================
# 1. UTILS: COORD TRANSFORM
# ==========================================
x_pi = 3.14159265358979324 * 3000.0 / 180.0
pi = 3.1415926535897932384626
a = 6378245.0
ee = 0.00669342162296594323

def gcj02_to_bd09(lng, lat):
    z = math.sqrt(lng * lng + lat * lat) + 0.00002 * math.sin(lat * x_pi)
    theta = math.atan2(lat, lng) + 0.000003 * math.cos(lng * x_pi)
    bd_lng = z * math.cos(theta) + 0.0065
    bd_lat = z * math.sin(theta) + 0.006
    return [bd_lng, bd_lat]

def bd09_to_gcj02(bd_lon, bd_lat):
    x = bd_lon - 0.0065
    y = bd_lat - 0.006
    z = math.sqrt(x * x + y * y) - 0.00002 * math.sin(y * x_pi)
    theta = math.atan2(y, x) - 0.000003 * math.cos(x * x_pi)
    gg_lng = z * math.cos(theta)
    gg_lat = z * math.sin(theta)
    return [gg_lng, gg_lat]

def wgs84_to_gcj02(lng, lat):
    if out_of_china(lng, lat):
        return [lng, lat]
    dlat = _transformlat(lng - 105.0, lat - 35.0)
    dlng = _transformlng(lng - 105.0, lat - 35.0)
    radlat = lat / 180.0 * pi
    magic = math.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = math.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * math.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return [mglng, mglat]

def _transformlat(lng, lat):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + \
          0.1 * lng * lat + 0.2 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lat * pi) + 40.0 *
            math.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * math.sin(lat / 12.0 * pi) + 320 *
            math.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

def _transformlng(lng, lat):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + \
          0.1 * lng * lat + 0.1 * math.sqrt(math.fabs(lng))
    ret += (20.0 * math.sin(6.0 * lng * pi) + 20.0 *
            math.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * math.sin(lng * pi) + 40.0 *
            math.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * math.sin(lng / 12.0 * pi) + 300.0 *
            math.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

def out_of_china(lng, lat):
    return not (lng > 73.66 and lng < 135.05 and lat > 3.86 and lat < 53.55)

def convert_by_type(lng, lat, type):
    if type == 'g2b': return gcj02_to_bd09(lng, lat)
    elif type == 'b2g': return bd09_to_gcj02(lng, lat)
    elif type == 'w2g': return wgs84_to_gcj02(lng, lat)
    elif type == 'g2w': return [lng, lat] # Placeholder for reverse if needed, or implement full
    else: return [lng, lat]

# ==========================================
# 2. DATA CONTAINER & PREPROCESSING
# ==========================================

longitudeMin = 116.09608
longitudeMax = 116.71040
latitudeMin = 39.69086
latitudeMax = 40.17647

# Convert bounds
longitudeMin, latitudeMin = convert_by_type(lng=longitudeMin, lat=latitudeMin, type="g2w")
longitudeMax, latitudeMax = convert_by_type(lng=longitudeMax, lat=latitudeMax, type="g2w")

divideBound = 5
widthSingle = 0.01 / math.cos(latitudeMin / 180 * math.pi) / divideBound
width = math.floor((longitudeMax - longitudeMin) / widthSingle)
heightSingle = 0.01 / divideBound
height = math.floor((latitudeMax - latitudeMin) / heightSingle)

def fill_speed(speed_data):
    date_range = pd.date_range(start="2018-08-01", end="2018-11-01", freq="1H")[:-1]
    speed_data = speed_data.resample(rule="1H").mean()
    
    missing_dates = []
    for date in date_range:
        if date in speed_data.index:
             if any(speed_data.loc[date].isna()):
                missing_dates.append(date)
        else:
             missing_dates.append(date) # Handle totally missing index

    print(f"Found {len(missing_dates)} dates with missing data")
    one_week = datetime.timedelta(days=7)
    
    for date in tqdm(missing_dates, 'Fill speed'):
        # Simple fill logic for demo stability
        if date - one_week in speed_data.index:
             speed_data.loc[date] = speed_data.loc[date - one_week]
        else:
             speed_data.loc[date] = speed_data.mean() # Fallback
    return speed_data.fillna(0)

import datetime

class AccidentDataset(Dataset):
    def __init__(self, k_order, network, node_attr, accident, weather, speed, 
                 sf_scaler=None, tf_scaler=None, ef_scaler=None):
        self.k_order = k_order
        self.network = network
        self.nodes = node_attr
        self.accident = accident
        self.weather = weather
        self.speed = speed
        self.sf_scaler = sf_scaler
        self.tf_scaler = tf_scaler
        self.ef_scaler = ef_scaler

    def __getitem__(self, sample_id):
        _, _, accident_time, node_id, target = self.accident.iloc[sample_id]
        
        # 1. Subgraph extraction
        neighbors = nx.single_source_shortest_path_length(self.network, node_id, cutoff=self.k_order)
        neighbors.pop(node_id, None)
        neighbors = [node_id] + sorted(neighbors.keys())
        
        sub_graph_nx = nx.subgraph(self.network, neighbors)
        relabel_map = {old_label: new_label for new_label, old_label in enumerate(neighbors)}
        sub_graph_nx = nx.relabel_nodes(sub_graph_nx, relabel_map)
        
        # --- FIX: XỬ LÝ TRƯỜNG HỢP KHÔNG CÓ CẠNH ---
        edges = list(sub_graph_nx.edges)
        if len(edges) > 0:
            edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
        else:
            # Nếu không có cạnh, tạo tensor rỗng shape [2, 0]
            edge_index = torch.empty((2, 0), dtype=torch.long)
        # -------------------------------------------

        # 2. Features Extraction
        # Sửa warning 'H' -> 'h'
        date_range = pd.date_range(end=accident_time.strftime("%Y%m%d %H"), freq="1h", periods=24)
        
        valid_dates = [d for d in date_range if d in self.speed.index]
        if len(valid_dates) < 24:
             selected_time = pd.DataFrame(0, index=date_range, columns=self.speed.columns)
        else:
             selected_time = self.speed.loc[date_range]

        selected_nodes = self.nodes.loc[neighbors]
        spatial_features = selected_nodes['spatial_features'].tolist()

        x_ids = np.floor((selected_nodes['XCoord'].values - longitudeMin) / widthSingle).astype(np.int64)
        y_ids = np.floor((selected_nodes['YCoord'].values - latitudeMin) / heightSingle).astype(np.int64)
        
        keys = [f'{y},{x}' for y, x in zip(y_ids, x_ids)]
        valid_keys = [k for k in keys if k in selected_time.columns]
        
        if not valid_keys:
             temporal_features = np.zeros((len(neighbors), 24))
        else:
            temp_vals = []
            for k in keys:
                if k in selected_time.columns:
                    temp_vals.append(selected_time[k].values)
                else:
                    temp_vals.append(np.zeros(24))
            temporal_features = np.array(temp_vals)

        if date_range[-1] in self.weather.index:
            weather_data = self.weather.loc[date_range[-1]].tolist()
        else:
            weather_data = [0] * len(self.weather.columns)

        external_features = weather_data + [accident_time.month, accident_time.day, accident_time.dayofweek,
                                            accident_time.hour, int(accident_time.dayofweek >= 5)]

        # Scaling
        if self.sf_scaler[0] is not None:
            spatial_features = (np.array(spatial_features) - self.sf_scaler[0]) / (self.sf_scaler[1] + 1e-5)
        if self.tf_scaler[0] is not None:
             temporal_features = (np.array(temporal_features) - self.tf_scaler[0]) / (self.tf_scaler[1] + 1e-5)
        if self.ef_scaler[0] is not None:
             external_features = (np.array(external_features) - self.ef_scaler[0]) / (self.ef_scaler[1] + 1e-5)

        x_spatial = torch.tensor(spatial_features).float()
        x_temporal = torch.tensor(temporal_features).float()
        x_external = torch.tensor(external_features).float().unsqueeze(0).repeat(len(neighbors), 1)
        
        x = torch.cat([x_spatial, x_temporal, x_external], dim=1)
        y = torch.tensor(target).long()

        target_mask = torch.zeros(len(neighbors), dtype=torch.bool)
        target_mask[0] = True
        data = Data(x=x, edge_index=edge_index, y=y, target_mask=target_mask)
        
        # --- FIX: TÍNH NUM_EDGES AN TOÀN ---
        num_edges = edge_index.shape[1] 
        # -----------------------------------
        
        data.edge_attr_dir = torch.zeros((num_edges, 1))
        data.edge_attr_ang = torch.zeros((num_edges, 1))
        data.edge_attr = torch.zeros((num_edges, 1))
        
        return data

    def __len__(self):
        return len(self.accident)

# Helper to get metadata (Mocking the missing h5 attributes)
def get_attribute_mock(nodes, speed, weather):
    # This is a simplified mock. In real usage, calculate mean/std from Training set only.
    sf_mean = 0 
    sf_std = 1
    tf_mean = 0
    tf_std = 1
    ef_mean = 0
    ef_std = 1
    return sf_mean, sf_std, tf_mean, tf_std, ef_mean, ef_std

def get_data_loaders(k_order, batch_size):
    # Paths (Ensure these exist in your Kaggle environment)
    network_path = r'/kaggle/input/dstgcn-dataset/beijing_roadnet.gpickle'
    node_attr_path = r'/kaggle/input/dstgcn-dataset/edges_data.h5'
    accident_path = r'/kaggle/input/dstgcn-dataset/accident.h5'
    weather_path = "/kaggle/input/dstgcn-dataset/weather.h5"
    speed_path = "/kaggle/input/dstgcn-dataset/all_grids_speed.h5"

    print("Loading data...")
    with open(network_path, 'rb') as f:
        network = pickle.load(f)
    nodes = pd.read_hdf(node_attr_path)
    weather = pd.read_hdf(weather_path)
    # Mocking fill_speed for speed (requires data)
    # speed = fill_speed(pd.read_hdf(speed_path)) 
    speed = pd.read_hdf(speed_path) # Assuming clean for now or implement fill_speed fully

    # Mock scaler stats
    sf_mean, sf_std, tf_mean, tf_std, ef_mean, ef_std = get_attribute_mock(nodes, speed, weather)

    dls = dict()
    for key in ['train', 'validate', 'test']:
        try:
            accident = pd.read_hdf(accident_path, key=key)
        except KeyError:
            print(f"Key {key} not found in accident.h5, skipping or using train")
            continue
            
        dataset = AccidentDataset(k_order, network, nodes, accident, weather, speed,
                                  sf_scaler=(sf_mean, sf_std),
                                  tf_scaler=(tf_mean, tf_std),
                                  ef_scaler=(ef_mean, ef_std))
        
        # Use PyG DataLoader
        dls[key] = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=(key=='train'), num_workers=0)
    
    return dls

# ==========================================
# 3. MODEL: TRAVELConv & TRAVELNet
# ==========================================

class TRAVELConv(MessagePassing):
    def __init__(self, in_channels: Union[int, Tuple[int, int]],
                 out_channels: int, nn: Callable, aggr: str = 'add',
                 root_weight: bool = True, bias: bool = True, **kwargs):
        super(TRAVELConv, self).__init__(aggr=aggr, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.nn = nn
        self.aggr = aggr

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        self.in_channels_l = in_channels[0]

        if root_weight:
            self.root = Parameter(torch.Tensor(in_channels[1], out_channels))
        else:
            self.register_parameter('root', None)

        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self.reset_parameters()

    def reset_parameters(self):
        reset(self.nn)
        if self.root is not None:
            uniform(self.root.size(0), self.root)
        zeros(self.bias)


    def forward(self, x: Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]], edge_index,
                edge_attr=None, size=None) -> torch.Tensor:
        if isinstance(x, torch.Tensor):
            x = (x, x)

        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=size)

        x_r = x[1]
        if x_r is not None and self.root is not None:
            out += torch.matmul(x_r, self.root)

        if self.bias is not None:
            out += self.bias
        return out


    def message(self, x_i, x_j, edge_attr):
        # Concatenate Node features of neighbor (x_j) and Edge features
        if edge_attr is not None:
            inputs = torch.cat([x_j, edge_attr], dim=1)
        else:
            inputs = x_j
        return self.nn(inputs)

class TRAVELNet(torch.nn.Module):
    def __init__(self, node_feature_dim, edge_dir_dim, edge_ang_dim, num_classes, dim=64, p_dropout=0.5):
        super(TRAVELNet, self).__init__()
        self.p = p_dropout
        convdim = 8
        
        # Node Encoder
        self.node_encoder = nn.Sequential(nn.Linear(node_feature_dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim))
        
        # Edge Encoders
        self.edge_encoder_dir = nn.Sequential(nn.Linear(edge_dir_dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim))
        self.edge_encoder_ang = nn.Sequential(nn.Linear(edge_ang_dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim))
        
        # Conv Branch 1 (Direction)
        nn1 = nn.Sequential(nn.Linear(dim + dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim), nn.LeakyReLU(), nn.Linear(dim, convdim))
        self.conv1 = TRAVELConv(dim, convdim, nn1)
        
        nn2 = nn.Sequential(nn.Linear(2*convdim + dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim), nn.LeakyReLU(), nn.Linear(dim, num_classes))
        self.conv2 = TRAVELConv(2*convdim, num_classes, nn2)
        
        self.bn1 = nn.BatchNorm1d(convdim*2)
        
        # Conv Branch 2 (Angle)
        nn1_2 = nn.Sequential(nn.Linear(dim + dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim), nn.LeakyReLU(), nn.Linear(dim, convdim))
        self.conv1_2 = TRAVELConv(dim, convdim, nn1_2)
        
        nn2_2 = nn.Sequential(nn.Linear(2*convdim + dim, dim), nn.LeakyReLU(), nn.Linear(dim, dim), nn.LeakyReLU(), nn.Linear(dim, num_classes))
        self.conv2_2 = TRAVELConv(2*convdim, num_classes, nn2_2)
        
        self.bn2 = nn.BatchNorm1d(num_classes*2)
        self.fc = nn.Linear(num_classes*2, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # Encode inputs
        x = self.node_encoder(x)
        
        # Assume component_dir and component_ang are attached to data in batching or preprocessing
        # If not present, create placeholders (Not ideal for performance but prevents crash)
        if hasattr(data, 'component_dir'):
            edge_attr_dir = self.edge_encoder_dir(data.component_dir)
        else:
            # Fallback for debugging
            edge_attr_dir = self.edge_encoder_dir(torch.zeros(edge_index.size(1), 2).to(x.device)) # Assumed dim 2
            
        if hasattr(data, 'component_ang'):
            edge_attr_ang = self.edge_encoder_ang(data.component_ang)
        else:
             edge_attr_ang = self.edge_encoder_ang(torch.zeros(edge_index.size(1), 2).to(x.device))

        # Block 1
        x1 = F.relu(self.conv1(x, edge_index, edge_attr_dir))
        x2 = F.relu(self.conv1_2(x, edge_index, edge_attr_ang))
        
        x_concat = torch.cat((x1, x2), axis=1)
        x_concat = self.bn1(x_concat)
        x_concat = F.dropout(x_concat, p=self.p, training=self.training)
        
        # Block 2
        x1_out = F.relu(self.conv2(x_concat, edge_index, edge_attr_dir))
        x2_out = F.relu(self.conv2_2(x_concat, edge_index, edge_attr_ang))
        
        x_final = torch.cat((x1_out, x2_out), axis=1)
        x_final = self.fc(x_final)
        
        return F.log_softmax(x_final, dim=1)

# ==========================================
# 4. TRAIN AND TEST LOOP
# ==========================================

def train(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    for batch in data_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        
        # Mock edge features (giữ nguyên logic cũ)
        batch.component_dir = torch.cat([batch.edge_attr, batch.edge_attr_dir], dim=1).float()
        batch.component_ang = torch.cat([batch.edge_attr, batch.edge_attr_ang], dim=1).float()
        
        out = model(batch)
        
        # --- FIX QUAN TRỌNG TẠI ĐÂY ---
        # Chỉ lấy output của các node trung tâm (target nodes)
        # out shape ban đầu: [Total_Nodes_in_Batch, 2] (ví dụ: 162, 2)
        # batch.y shape: [Batch_Size] (ví dụ: 32)
        
        pred_nodes = out[batch.target_mask] # Shape sẽ thành [32, 2]
        
        loss = F.nll_loss(pred_nodes, batch.y)
        # ------------------------------
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)
    
@torch.no_grad()
def test(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []
    y_scores = []
    
    for batch in data_loader:
        batch = batch.to(device)
        batch.component_dir = torch.cat([batch.edge_attr, batch.edge_attr_dir], dim=1).float()
        batch.component_ang = torch.cat([batch.edge_attr, batch.edge_attr_ang], dim=1).float()
        
        out = model(batch)
        
        # --- FIX ÁP DỤNG MASK CHO TEST ---
        pred_nodes = out[batch.target_mask] # Lọc ra các node target
        
        pred = pred_nodes.max(1)[1]
        y_scores_batch = torch.exp(pred_nodes)[:, 1] # Lấy xác suất lớp 1
        # ---------------------------------
        
        y_true.append(batch.y.cpu().numpy())
        y_pred.append(pred.cpu().numpy())
        y_scores.append(y_scores_batch.cpu().numpy()) 

    y_true = np.concatenate(y_true)
    y_pred = np.concatenate(y_pred)
    y_scores = np.concatenate(y_scores)
    
    # (Phần tính toán metrics bên dưới giữ nguyên)
    f1 = f1_score(y_true, y_pred, average='binary')
    acc = (y_true == y_pred).sum() / len(y_true)
    try:
        auc = roc_auc_score(y_true, y_scores)
        ap = average_precision_score(y_true, y_scores)
    except ValueError:
        auc = 0
        ap = 0

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    pcc, _ = pearsonr(y_true, y_pred)

    return f1, acc, ap, auc, mae, rmse, pcc

def train_loop(model, loaders, optimizer, num_epochs, model_name='TRAVEL'):
    print(f"Start training {model_name}...")
    best_f1 = 0
    
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, loaders['train'], optimizer)
        
        # Evaluate on Val (or Test directly if Val not present)
        eval_key = 'validate' if 'validate' in loaders else 'test'
        val_res = test(model, loaders[eval_key])
        f1, acc, map_score, auc, mae, rmse, pcc = val_res
        
        log = 'Epoch: {:03d}, Loss: {:.4f}, F1: {:.4f}, Acc: {:.4f}, AUC: {:.4f}, MAE: {:.4f}, RMSE: {:.4f}, PCC: {:.4f}'
        print(log.format(epoch, train_loss, f1, acc, auc, mae, rmse, pcc))

# ==========================================
# 5. MAIN EXECUTION
# ==========================================

if __name__ == "__main__":
    # Settings
    BATCH_SIZE = 32
    K_ORDER = 1 # Neighbor depth
    NUM_EPOCHS = 20
    HIDDEN_DIM = 64
    NUM_CLASSES = 2 # 0 or 1
    
    # 1. Load Data
    try:
        loaders = get_data_loaders(K_ORDER, BATCH_SIZE)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please ensure dataset files are in /kaggle/input/dstgcn-dataset/")
        sys.exit(1)
    
    if 'train' not in loaders:
        print("No training data found. Exiting.")
        sys.exit(1)

    # 2. Inspect a batch to determine input dimensions dynamically
    sample_batch = next(iter(loaders['train']))
    node_feat_dim = sample_batch.x.size(1)
    # Edge attr calculated as concatenation of [edge_attr, edge_attr_dir/ang]
    # In Dataset we made placeholders of size 1. So 1+1 = 2 dimensions.
    edge_dim = 2 
    
    # 3. Initialize Model
    model = TRAVELNet(
        node_feature_dim=node_feat_dim,
        edge_dir_dim=edge_dim,
        edge_ang_dim=edge_dim,
        num_classes=NUM_CLASSES,
        dim=HIDDEN_DIM
    ).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

    # 4. Run Training
    train_loop(model, loaders, optimizer, NUM_EPOCHS)

Loading data...
Start training TRAVEL...
Epoch: 001, Loss: 0.4593, F1: 0.8417, Acc: 0.8422, AUC: 0.9246, MAE: 0.1578, RMSE: 0.3973, PCC: 0.6844
Epoch: 002, Loss: 0.3828, F1: 0.8511, Acc: 0.8578, AUC: 0.9360, MAE: 0.1422, RMSE: 0.3771, PCC: 0.7186
Epoch: 003, Loss: 0.3634, F1: 0.8127, Acc: 0.8344, AUC: 0.9380, MAE: 0.1656, RMSE: 0.4070, PCC: 0.6874
Epoch: 004, Loss: 0.3586, F1: 0.8768, Acc: 0.8758, AUC: 0.9383, MAE: 0.1242, RMSE: 0.3524, PCC: 0.7517
Epoch: 005, Loss: 0.3447, F1: 0.8692, Acc: 0.8742, AUC: 0.9469, MAE: 0.1258, RMSE: 0.3547, PCC: 0.7506
Epoch: 006, Loss: 0.3459, F1: 0.8716, Acc: 0.8695, AUC: 0.9441, MAE: 0.1305, RMSE: 0.3612, PCC: 0.7395
Epoch: 007, Loss: 0.3364, F1: 0.8743, Acc: 0.8641, AUC: 0.9461, MAE: 0.1359, RMSE: 0.3687, PCC: 0.7379
Epoch: 008, Loss: 0.3442, F1: 0.8662, Acc: 0.8680, AUC: 0.9405, MAE: 0.1320, RMSE: 0.3634, PCC: 0.7362
Epoch: 009, Loss: 0.3445, F1: 0.8711, Acc: 0.8641, AUC: 0.9471, MAE: 0.1359, RMSE: 0.3687, PCC: 0.7325
Epoch: 010, Loss: 0.3318, F1: 0.